<a href="https://colab.research.google.com/github/salmanullahbaig/crypto_trading_bot/blob/main/Bot_testing_with_SL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python_binance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 65 kB 2.9 MB/s 
     |████████████████████████████████| 290 kB 13.8 MB/s 
     |████████████████████████████████| 106 kB 64.5 MB/s 
     |████████████████████████████████| 749 kB 57.3 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2022.6.2
    Uninstalling regex-2022.6.2:
      Successfully uninstalled regex-2022.6.2


In [ ]:
!pip install pandas_ta  ta 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 115 kB 6.4 MB/s 
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=1a8de215b7ba251640a49bf2ffb9de1eaa24cf375ed9a03588492414290a6406
  Stored in directory: /root/.cache/pip/wheels/0b/81/f0/cca85757840e4616a2c6b9fe12569d97d324c27cac60724c58
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29104 sha256=698176c31d6d0938267ab86d22337c79605be2c0efaafc8197e12581b1089d79
  Stored in directory: /root/.cache/pip/wheels/31/31/f1/f2ff471bbc5b84a4b973698ceecdd453ae043971791adc3431
Successfully built pandas-ta ta


In [ ]:
import sys
import os
#sys.path.append(os.path.abspath("/content"))
#from util1 import *
from util import *
from tensorflow.keras.models import load_model
import time
import pandas as pd
from sklearn.preprocessing import scale as scaler
import numpy as np
from binance.client import Client
from binance.enums import HistoricalKlinesType
import pandas as pd
import pandas_ta as pta
from ta import add_all_ta_features
import requests


columns = columns[0].split("+")
model = load_model('model.h5')

#model = load_model('data/model.h5')
#load_model(filepath)

class Get_data:
  def __init__(self , client ,interval, coin = "XRPUSDT", start = "1 Jan, 2021", end = "12 May, 2021"):
    self.client = client
    self.interval = interval
    self.coin = coin
    self.start = start
    self.end = end
    #self.df = get_data(clint,interval, coin , start , end )
  def get_data(self):
      columns = [ "Open time", "open","high", " low"  ,"close", "volume" , 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']
      #columns = ["Open time", "open","high", " low"  ,"close", "volume"]
      candlesticks = self.client.get_historical_klines(self.coin, self.interval,self.start , self.end, klines_type  = HistoricalKlinesType.SPOT  )
      df = pd.DataFrame(candlesticks, columns = columns)
      columns = [ "Open time", "open","high", " low"  ,"close", "volume" , 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume']
      df = df[columns]
      df["Open time"] = pd.to_datetime(df["Open time"] ,unit='ms')
      return df
#Get_data(client,Client.KLINE_INTERVAL_30MINUTE, "XRPUSDT", "1 Jan, 2021", "12 May, 2021").get_data()

class addIndictors:
  def __init__(self, df):
    self.df = df
    df[['open', 'high', ' low', 'close', 'volume']] =df[['open', 'high', ' low', 'close', 'volume']].astype('float')
  def add_indictors(self,df , moving_a = [ 2,7,10,20, 30,50,100, 200, 300], rsi_r = [10,14,20]):
    for range in moving_a:
      df.ta.sma(length=range, append = True)
      df.ta.ema(length=range, append = True)
    for range in rsi_r:
      df.ta.rsi(length=range, append = True)
    return df
  def get_indictors(self):
    df = add_all_ta_features(self.df,  open="open", high="high", low=" low", close="close", volume="volume", fillna=True)
    df = self.add_indictors(df)
    return df



def prepare_data(df):

  data = df[columns]
  data = data.astype(float)
  values_after_scale = scaler(data)
  data= pd.DataFrame(values_after_scale,columns=data.columns)
  return data

#data = prepare_data(df)

In [ ]:
api_key = "zRbnRTq5K7R8TFoF2W0wcfx4Jz9al3MYTrzxHbbpWjPZhiq7fLzsKUpcEc63jHyG"
secret_key = "XVqunTEKoLMRz9FGuXbo1ZXbLnhTdQnRs0St3d5CrHz59kC9XvFV3E16L2f9kQWC"

api_key = "UGPIJJEE4CKCXpfbewELMRYl7UnyiTotoWlSL9kKY0OlMgqBHHHh8QOAXX2S2uXY"
secret_key = "BjgpPTvnGKwxyKlK96iVfr8HhldyITK687i8it6aWjW27Og8mA7F2qAngdQOfkNA"

client = Client(api_key, secret_key, requests_params = {"timeout":100 } )
print("Getting data ...")
#df = Get_data( client,interval = Client.KLINE_INTERVAL_15MINUTE, coin = "BTCUSDT", start = "27 Jan, 2022", end = "16 Dec, 2022").get_data()
# print(df.info())
# #client.API_URL = 'https://testnet.binance.vision/api'
# df = addIndictors(df).get_indictors()

df = Get_data(client,interval = Client.KLINE_INTERVAL_15MINUTE, coin = "BTCUSDT", start = "1 Oct, 2022", end = "16 Dec, 2025").get_data()#[-300:]
df = addIndictors(df).get_indictors()
data = prepare_data(df)
#label = [np.argmax(x) for x in model.predict(data[-1:])][0]
df

Getting data ...


/usr/local/lib/python3.7/dist-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/usr/local/lib/python3.7/dist-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


,Open time,open,high,low,close,volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,...,EMA_50,SMA_100,EMA_100,SMA_200,EMA_200,SMA_300,EMA_300,RSI_10,RSI_14,RSI_20
0,2022-10-01 00:00:00,19422.61,19466.98,19407.00,19459.44,2425.94976,47153777.71247580,30232,1243.79105000,24177158.52739130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-10-01 00:15:00,19460.16,19484.00,19430.00,19452.68,1829.54051,35605396.72972380,27011,936.49672000,18226334.82651930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-10-01 00:30:00,19451.71,19454.57,19351.02,19358.45,2755.62211,53439688.22280550,42981,1250.45530000,24250960.17705870,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-10-01 00:45:00,19358.45,19402.89,19354.85,19402.88,1640.53640,31795192.52672930,28416,820.79451000,15908362.15202530,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-10-01 01:00:00,19401.84,19405.58,19371.26,19389.08,1569.76074,30439629.72269550,24237,779.85286000,15122926.07271240,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2710,2022-10-29 05:30:00,20722.58,20723.81,20677.72,20682.42,2297.17962,47554274.62812540,57197,1132.40074000,23442422.19391770,...,20626.474542,20477.1582,20556.628600,20505.06225,20483.618106,20541.432267,20390.652642,49.495282,51.781710,53.688990
2711,2022-10-29 05:45:00,20682.95,20696.62,20620.81,20655.56,2673.36103,55210488.83630500,66415,1288.09214000,26601649.74129730,...,20627.615148,20481.1943,20558.587637,20504.56340,20485.328971,20542.915367,20392.412823,43.952202,47.584659,50.720863
2712,2022-10-29 06:00:00,20655.56,20684.89,20647.89,20673.31,1341.16896,27720462.16735400,43632,656.90058000,13577566.69371040,...,20629.407103,20485.5810,20560.859367,20504.19475,20487.199429,20544.510667,20394.279250,48.210888,50.443223,52.545770
2713,2022-10-29 06:15:00,20673.31,20687.65,20662.72,20684.67,1218.89311,25199736.00861730,40531,635.21251000,13132984.64038050,...,20631.574276,20489.8729,20563.311063,20503.66530,20489.164310,20545.844267,20396.208757,50.865733,52.238503,53.700838


In [ ]:
# import datetime


# for trade in client.get_my_trades(symbol="BTCUSDT"):
#   dobj = datetime.datetime.fromtimestamp(trade['time']/1000)
#   if dobj.month == 9 and  dobj.day > 6:
#     trade['time'] = dobj
#     print(trade)


In [ ]:
#client.get_deposit_history(coin="USDT")

In [ ]:
#dobj.month , dobj.day

In [ ]:
#df

In [ ]:
def get_score(buying_p, selling_p, SL, output = 0):
    df['low'] = df[' low']
    time = df['Open time']
    holding = 0
    funds = initial_amount =10000
    net =model #neat.nn.FeedForwardNetwork.create(genome, config)

    tax = 0
    Transaction = 0

    already_buyed = False
    already_selled = False
    stop_loss_trigger = False
    buying_probability = buying_p

    #varibles for going short
    number_of_share_use_for_short = 0
    buying_price = 0
    total_amount_for_short = 0
    profit_from_short = 0
    already_buyed = False
    already_selled = True 
    buyed_at = 0
    sl_count = 0
    total_trades = 0

    f = open("Test_results.txt", "a")
    x_train=data
    for i in range(300, len(data)):
        signal =model.predict(data[i:i+1], verbose = 0)
        label = np.argmax(signal)  #Label code :
        out_put = " " 

  
        if label == 1  and already_buyed ==False and float(df.open[i])  < float(df.SMA_7.iloc[i:i+1]):
            #print(signal)
            bp = float([x for x in signal][0][1])
            #print("buying", bp)
            if  bp> buying_probability:
                if output:
                  out_put = " Signal = Opening Long ,\n Total coins  before Opening Long "+ str(holding) + " and total amount to invest = " + str(funds) + " \n"
                holding += (funds*(1 - tax))/df.open[i] #Total amoutn invested - text  then divide
                funds = 0
                if output:
                  out_put = out_put + "Time: "+ str(time[i])+"  Open price: "+ str(df.open[i]) + "\n"
                  out_put =out_put + "Total coins after Opening LOng " +  str(holding) + " and total amount to invest = " + str(funds) + "\n"
                open_at = df.open[i]
                already_buyed =True
                already_selled = False
                buyed_at = float(df.open[i])
                Transaction =Transaction + 1
                total_trades += 1

                #print("Date: ", time[i] ,"  Open Price :  ",df.open[i], "Action Buying ..")
                if output:
                  out_put =out_put + "Date: " + str(time[i]) +"  Open Price :  " + str(df.open[i]) +"Action Buying .." 
                  out_put =out_put + " \n*******************************************"
                  #print("Buying ...")
                  #print('****************************************')
                  if output:
                    print(out_put)
                  #f.write(out_put)
        if label == 0 and  float(df.close[i]) > buyed_at+50  and already_selled == False:#  and float(df.close[i])  > float(df.SMA_7.iloc[-1]):
            sp =  float([x for x in signal][0][0])
            #print('selling', sp)
            if  sp> selling_p:
                if output:
                  out_put =out_put + " Signal = Closing Long  \n Total coins  before Closing Long "+ str(holding) + " and total amount to invest = " + str(funds) + " \n"
                funds = holding * df.close[i] * (1-tax)
                holding = 0
                hasSold = True
                already_selled = True 
                already_buyed =False
                if output:
                  out_put =out_put + "Total coins after Closing LOng " +  str(holding) + " and total amount to invest = " + str(funds) + "\n"
                Transaction  =Transaction+ 1
                #print("Selling ...")
                if output:
                  out_put =out_put + "Date: "+ str(time[i] ) +  "  Closing Price :  "+ str(df.close[i]) + "Action Selling .."
                  out_put =out_put + " \n*******************************************"
                  if output:
                    print(out_put)
                  #print("Date: ", time[i] , "  Closing Price :  ",df.close[i], "Action Selling ..")
                  #print('****************************************')
                  #f.write(out_put)
        if already_buyed == True:
            #print(float(df.low[i]))
            if  float(df.low[i]) < buyed_at -SL :
                if output:
                  out_put =out_put + " Signal = Emergency loss activated  \n Total coins  before Closing Long "+ str(holding) + " and total amount to invest = " + str(funds) + " \n"
                open_at = open_at - SL
                sl_count += 1
                funds = holding * open_at * (1-tax)
                holding = 0
                hasSold = True
                already_selled = True 
                already_buyed =False
                if output:
                  out_put =out_put + "Total coins after Closing LOng " +  str(holding) + " and total amount to invest = " + str(funds) + "\n"
                Transaction  =Transaction+ 1
                #print("Selling ...")
                if output:
                  out_put =out_put + "Date: "+ str(time[i] ) +  "  Closing Price :  "+ str(open_at) + "Action Selling .."
                  out_put =out_put + " \n*******************************************"
                  if output:
                    print(out_put)
                
                #f.write(out_put)
    score = funds + holding*df.close[i]
    #print(df.close[i])
    print("SL hit = ", sl_count)
    print("Total trades = ", total_trades)
    #detail = str(f" Funds: {funds}, Hold ETH: {holding}, Current score: {score}, Current Score in ETH:{score/df.close[i]}")#, "Signal = ", signal)
    return score


In [ ]:
total = get_score(0.51, 0.51, 200, output= 1)
total
print(total)
((total - 10000)/10000* 100)

 Signal = Opening Long ,
 Total coins  before Opening Long 0 and total amount to invest = 10000 
Time: 2022-10-04 04:15:00  Open price: 19561.39
Total coins after Opening LOng 0.5112111153655237 and total amount to invest = 0
Date: 2022-10-04 04:15:00  Open Price :  19561.39Action Buying .. 
*******************************************
  Signal = Closing Long  
 Total coins  before Closing Long 0.5112111153655237 and total amount to invest = 0 
Total coins after Closing LOng 0 and total amount to invest = 10028.586925571242
Date: 2022-10-04 05:15:00  Closing Price :  19617.31Action Selling .. 
*******************************************
 Signal = Opening Long ,
 Total coins  before Opening Long 0 and total amount to invest = 10028.586925571242 
Time: 2022-10-04 10:15:00  Open price: 19901.56
Total coins after Opening LOng 0.5039095892769834 and total amount to invest = 0
Date: 2022-10-04 10:15:00  Open Price :  19901.56Action Buying .. 
*******************************************
  Sign

12.598847906625487

In [ ]:
total = get_score(0.538, 0.518, 200, output= 1)
total

 Signal = Opening Long ,
 Total coins  before Opening Long 0 and total amount to invest = 10000 
Time: 2022-10-04 12:45:00  Open price: 19911.89
Total coins after Opening LOng 0.5022124971562217 and total amount to invest = 0
Date: 2022-10-04 12:45:00  Open Price :  19911.89Action Buying .. 
*******************************************
  Signal = Closing Long  
 Total coins  before Closing Long 0.5022124971562217 and total amount to invest = 0 
Total coins after Closing LOng 0 and total amount to invest = 10040.654101644795
Date: 2022-10-04 13:00:00  Closing Price :  19992.84Action Selling .. 
*******************************************
 Signal = Opening Long ,
 Total coins  before Opening Long 0 and total amount to invest = 10040.654101644795 
Time: 2022-10-04 17:45:00  Open price: 19951.77
Total coins after Opening LOng 0.5032462834948876 and total amount to invest = 0
Date: 2022-10-04 17:45:00  Open Price :  19951.77Action Buying .. 
*******************************************
  Sign

10557.41135828761

In [ ]:
27/68

0.39705882352941174

In [ ]:
total = get_score(0.51, 0.51, 150, output= 1) #SL hit =  27 Total trades =  68 , 10759. , 0.51, 0.51, 125,
total

 Signal = Opening Long ,
 Total coins  before Opening Long 0 and total amount to invest = 10000 
Time: 2022-10-04 04:15:00  Open price: 19561.39
Total coins after Opening LOng 0.5112111153655237 and total amount to invest = 0
Date: 2022-10-04 04:15:00  Open Price :  19561.39Action Buying .. 
*******************************************
  Signal = Closing Long  
 Total coins  before Closing Long 0.5112111153655237 and total amount to invest = 0 
Total coins after Closing LOng 0 and total amount to invest = 10028.586925571242
Date: 2022-10-04 05:15:00  Closing Price :  19617.31Action Selling .. 
*******************************************
 Signal = Opening Long ,
 Total coins  before Opening Long 0 and total amount to invest = 10028.586925571242 
Time: 2022-10-04 10:15:00  Open price: 19901.56
Total coins after Opening LOng 0.5039095892769834 and total amount to invest = 0
Date: 2022-10-04 10:15:00  Open Price :  19901.56Action Buying .. 
*******************************************
  Sign

11039.20947334011

In [ ]:
#total =67880
((total - 10000)/10000* 100)

10.3920947334011

In [ ]:
# buying_probabilits = [ 0.531 ,  0.533, 0.535 , 0.537 , 0.538 ] 
# Selling_probabilits = [ 0.512 ,  0.513, 0.514 , 0.516 , 0.518 , 0.519]
buying_probabilits = [ 0.51 ,  0.52, 0.53 , 0.54 , 0.55 , 0.56 , 0.57 , 0.58 , 0.59] 
Selling_probabilits = [ 0.51 ,  0.52, 0.53 , 0.54 , 0.55 , 0.56 , 0.57 , 0.58 , 0.59]


SL =  [100, 150, 200]
results = []
for sl in SL:
    for bp in buying_probabilits:
        for sp in Selling_probabilits:
            result = { "Buying Probability ": bp , " Selling Probability  ":  sp , "SL": sl , "Score  ":get_score(bp, sp, sl)}
            results.append(result)
            print(result)

In [ ]:
sorted(results,key=lambda x:x['Score  '], reverse= True)

In [ ]:
(12784 - 10000)/10000 * 100

In [ ]:
SL hit =  1020
Total trades =  2073
{'Buying Probability ': 0.51, ' Selling Probability  ': 0.51, 'SL': 200, 'Score  ': 73429.96232625844}

SL hit =  886
Total trades =  1786
{'Buying Probability ': 0.52, ' Selling Probability  ': 0.51, 'SL': 200, 'Score  ': 76438.80967552753}

SL hit =  624
Total trades =  1214
70822.2


In [ ]:
[{'Buying Probability ': 0.51,
  ' Selling Probability  ': 0.51,
  'SL': 150,
  'Score  ': 10731.960349060224},
 {'Buying Probability ': 0.51,
  ' Selling Probability  ': 0.51,
  'SL': 100,
  'Score  ': 10649.940934715318},
 {'Buying Probability ': 0.52,
  ' Selling Probability  ': 0.51,
  'SL': 100,
  'Score  ': 10566.063204400542},
 {'Buying Probability ': 0.58,
  ' Selling Probability  ': 0.51,
  'SL': 100,
  'Score  ': 10537.297016917084},
 {'Buying Probability ': 0.59,
  ' Selling Probability  ': 0.51,
  'SL': 100,
  'Score  ': 10460.644175152102},
 {'Buying Probability ': 0.52,
  ' Selling Probability  ': 0.51,
  'SL': 150,
  'Score  ': 10404.688729077752},
 {'Buying Probability ': 0.56,
  ' Selling Probability  ': 0.51,
  'SL': 100,
  'Score  ': 10380.901236938014},
 {'Buying Probability ': 0.57,
  ' Selling Probability  ': 0.51,
  'SL': 100,
  'Score  ': 10379.67034437062},
 {'Buying Probability ': 0.58,
  ' Selling Probability  ': 0.51,
  'SL': 150,
  'Score  ': 10376.420075275533},
 {'Buying Probability ': 0.58,
  ' Selling Probability  ': 0.51,
  'SL': 200,
  'Score  ': 10374.656454514541},
 {'Buying Probability ': 0.59,
  ' Selling Probability  ': 0.51,
  'SL': 150,
  'Score  ': 10367.251778973168},
 {'Buying Probability ': 0.51,
  ' Selling Probability  ': 0.51,
  'SL': 200,
  'Score  ': 10329.84324690002},
 {'Buying Probability ': 0.53,
  ' Selling Probability  ': 0.51,
  'SL': 100,
  'Score  ': 10316.599266942672},
 {'Buying Probability ': 0.59,
  ' Selling Probability  ': 0.51,
  'SL': 200,
  'Score  ': 10307.920402764388},
 {'Buying Probability ': 0.56,
  ' Selling Probability  ': 0.51,
  'SL': 150,
  'Score  ': 10307.052373164403},
 {'Buying Probability ': 0.57,
  ' Selling Probability  ': 0.51,
  'SL': 150,
  'Score  ': 10304.625108483458},
 {'Buying Probability ': 0.57,
  ' Selling Probability  ': 0.51,
  'SL': 200,
  'Score  ': 10225.101152555717},
 {'Buying Probability ': 0.55,
  ' Selling Probability  ': 0.51,
  'SL': 150,
  'Score  ': 10214.505195799264},
 {'Buying Probability ': 0.52,
  ' Selling Probability  ': 0.51,
  'SL': 200,
  'Score  ': 10203.251360253476},
 {'Buying Probability ': 0.55,
  ' Selling Probability  ': 0.51,
  'SL': 100,
  'Score  ': 10190.196565748387},
 {'Buying Probability ': 0.56,
  ' Selling Probability  ': 0.51,
  'SL': 200,
  'Score  ': 10141.801614669272},
 {'Buying Probability ': 0.54,
  ' Selling Probability  ': 0.51,
  'SL': 150,
  'Score  ': 10106.946951988137},
 {'Buying Probability ': 0.54,
  ' Selling Probability  ': 0.51,
  'SL': 100,
  'Score  ': 10088.900071531438},
 {'Buying Probability ': 0.53,
  ' Selling Probability  ': 0.51,
  'SL': 150,
  'Score  ': 10083.317674124186},
 {'Buying Probability ': 0.53,
  ' Selling Probability  ': 0.51,
  'SL': 200,
  'Score  ': 10007.944831334664},
 {'Buying Probability ': 0.55,
  ' Selling Probability  ': 0.51,
  'SL': 200,
  'Score  ': 9972.616185921646},